In [1]:
from pathlib import Path
from FISHClass.datasets import MYCN
from torch.utils.data import DataLoader
import yaml
from baseline import models
from FISHClass.utils.evaluation import get_top_model
from FISHClass.evaluation.evaluate_test_set import predict_test, predict_test_baseline
from FISHClass.models import CombinedModel
from FISHClass.utils.device import best_gpu
import os
results_h5s = list(Path("/home/simon_g/src/results/EVALUATION").glob("*.h5"))

In [2]:
DATASET = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/MYCN_SpikeIn/results/h5/trainset_S6_S12_S29_S4_S19.h5"
OUT = "/home/simon_g/src/eval/test_set"
with open("/home/simon_g/src/FISHClass/evaluation/model_evaluation.yaml") as f:

    yaml_data = yaml.load(f, Loader=yaml.FullLoader)  

trained_model_paths = yaml_data["trained_model_paths"]

models_to_test = ["AreaModel", "CombinedModel", "ClassificationCNN", "FeaturespaceClassifier", "WeightedFeaturespaceClassifier", "SpotdetectionModel"]
combined_classifier_to_test = ["LSTMClassifier", "BasicClassifier"]
 
for test_model in models_to_test:

    if test_model == "AreaModel":
        model = getattr(models, test_model)(**yaml_data["AreaModel_kwargs"])
        results = predict_test_baseline(model, DATASET, dataset_kwargs={"norm_type": None, "transform": None}, save2h5=True, save_path=os.path.join(OUT, f"{test_model}_results.h5"))

    elif test_model == "SpotdetectionModel":

        model = getattr(models, test_model)(**yaml_data["SpotdetectionModel_kwargs"])
        results = predict_test_baseline(model, DATASET, dataset_kwargs={"norm_type": None, "transform": None}, save2h5=True, save_path=os.path.join(OUT, f"{test_model}_results.h5"))
        
    elif test_model == "CombinedModel":
        
        for classifier in combined_classifier_to_test:
        
            model = CombinedModel(fasterrcnn_path=trained_model_paths["FasterRCNNModel"], classification_path=trained_model_paths[classifier])
            results= predict_test(model, DATASET, device=best_gpu(), batch_size=16, dataset_kwargs={"norm_type": None, "transform": None}, save2h5=True, save_path=os.path.join(OUT, f"{test_model}{classifier}_results.h5"))
                    
    elif test_model == "FeaturespaceClassifier" or test_model == "WeightedFeaturespaceClassifier":
        
        model = get_top_model(trained_model_paths[test_model])
        results= predict_test(model, DATASET, batch_size=16, device=best_gpu(), dataset_kwargs={"norm_type": model.cnn_model.norm_type, "transform": None, "double_return": True}, save2h5=True, save_path=os.path.join(OUT, f"{test_model}_results.h5"))

    else:
        
        model = get_top_model(trained_model_paths[test_model])
        results = predict_test(model, DATASET, device=best_gpu(), dataset_kwargs={"norm_type": model.norm_type, "transform": None, "double_return": False}, save2h5=True, save_path=os.path.join(OUT, f"{test_model}_results.h5"))


100%|██████████| 32/32 [00:04<00:00,  6.59it/s]


Loaded FasterRCNN from /home/simon_g/src/results/trained_models/FASTER-R-CNN/FasterRCNN_20220914_192241464.pt
Loaded LSTMClassifier from /home/simon_g/src/results/trained_models/LSTM_MODELS/LSTMClassifier_20220915_225441081.pt
Using cuda:1 for calculation


100%|██████████| 63/63 [00:33<00:00,  1.89it/s]


Loaded FasterRCNN from /home/simon_g/src/results/trained_models/FASTER-R-CNN/FasterRCNN_20220914_192241464.pt
Loaded BasicClassifier from /home/simon_g/src/results/trained_models/LINEAR_MODELS/BasicClassifier_20220917_094243851.pt
Using cuda:2 for calculation


100%|██████████| 63/63 [00:31<00:00,  2.01it/s]


...

100%|██████████| 32/32 [00:05<00:00,  5.72it/s]


Loaded ClassificationCNN from /home/simon_g/src/results/trained_models/CNN_MODELS/ClassificationCNN_20220916_095919276.pt
Loaded FasterRCNN from /home/simon_g/src/results/trained_models/FASTER-R-CNN/FasterRCNN_20220914_192241464.pt
Loaded FeaturespaceClassifier from /home/simon_g/src/results/trained_models/FEATURESPACE_MODELS/FeaturespaceClassifier_20220917_020128922.pt
Using cuda:2 for calculation


100%|██████████| 63/63 [00:31<00:00,  1.98it/s]


Loaded ClassificationCNN from /home/simon_g/src/results/trained_models/CNN_MODELS/ClassificationCNN_20220916_095919276.pt
Loaded FasterRCNN from /home/simon_g/src/results/trained_models/FASTER-R-CNN/FasterRCNN_20220914_192241464.pt
Loaded LSTMClassifier from /home/simon_g/src/results/trained_models/LSTM_MODELS/LSTMClassifier_20220915_225441081.pt
Loaded WeightedFeaturespaceClassifier from /home/simon_g/src/results/trained_models/WEIGHTEDFEATURESPACE_MODELS/WeightedFeaturespaceClassifier_20220918_000053199.pt
Using cuda:1 for calculation


100%|██████████| 32/32 [00:06<00:00,  4.66it/s]


In [3]:
import h5py
import numpy as np

files = list(Path("/home/simon_g/src/eval/test_set").glob("*.h5"))
print(files)

[PosixPath('/home/simon_g/src/eval/test_set/SpotdetectionModel_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/AreaModel_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/FeaturespaceClassifier_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/CombinedModelLSTMClassifier_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/WeightedFeaturespaceClassifier_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/CombinedModelBasicClassifier_results.h5'), PosixPath('/home/simon_g/src/eval/test_set/ClassificationCNN_results.h5')]


In [38]:
for file in files: 
    
    with h5py.File(file) as fout:
        
        target = np.array(fout["TARGET"])
        pred = np.array(fout["PRED"])
        
    TP = len(np.where(np.logical_and(target == 1, pred == 1))[0])
    TN = len(np.where(np.logical_and(target == 0, pred == 0))[0])
    FP = len(np.where(np.logical_and(target == 0, pred == 1))[0])
    FN = len(np.where(np.logical_and(target == 1, pred == 0))[0])
    SPEC = np.round(TN / (TN + FP)*100,2)
    
    # Precision = TP / (TP + FP) 
    # Recall = TP / (TP + FN) 
    # F1 Score = 2 * Precision * Recall / (Precision + Recall)
    
    precision = np.round(TP/(TP+FP)*100,2)
    recall = np.round(TP/(TP+FN)*100,2)
    F1 = np.round(2*precision*recall/(precision+recall),2)
    
    print(f"{file.stem:<40}", " PRECISION: ", f"{precision:<8}", "RECALL/SENSITIVITY: ", f"{recall:<8}", "F1: ", f"{F1:<8}", "SPECIFICITY: ", f"{SPEC:<8}")

SpotdetectionModel_results                PRECISION:  80.74    RECALL/SENSITIVITY:  44.67    F1:  57.52    SPECIFICITY:  89.84   
AreaModel_results                         PRECISION:  92.02    RECALL/SENSITIVITY:  92.21    F1:  92.11    SPECIFICITY:  92.38   
FeaturespaceClassifier_results            PRECISION:  100.0    RECALL/SENSITIVITY:  97.34    F1:  98.65    SPECIFICITY:  100.0   
CombinedModelLSTMClassifier_results       PRECISION:  96.51    RECALL/SENSITIVITY:  96.31    F1:  96.41    SPECIFICITY:  96.68   
WeightedFeaturespaceClassifier_results    PRECISION:  99.39    RECALL/SENSITIVITY:  99.59    F1:  99.49    SPECIFICITY:  99.41   
CombinedModelBasicClassifier_results      PRECISION:  81.6     RECALL/SENSITIVITY:  89.96    F1:  85.58    SPECIFICITY:  80.66   
ClassificationCNN_results                 PRECISION:  99.18    RECALL/SENSITIVITY:  99.39    F1:  99.28    SPECIFICITY:  99.22   
